In [1]:
#import library
import nltk
import string
import re
import pandas as pd
import numpy as np

 #import stopword
from nltk.corpus import stopwords 
stopwords_indonesia = stopwords.words('indonesian')
 
#import sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer


In [2]:
#read data
def load_data():
    data = pd.read_csv('mandalika21032022_hapususer.csv')
    return data

#Pembuatan dataframe
df_twit = load_data()

#menampilkan data teratas
df_twit.head(10)
#len(df_twit)

,teks
0,"b""RT @Rizmaya__: Semula Mencibir, Kini Terpana..."
1,b'@Diniandik @lucywiryono Sabar mbak gitu dah ...
2,b'RT @aniesbaswedan: Hujan deras tidak menyuru...
3,"b'Dari Mandalika, Presiden @jokowi dan Ibu Iri..."
4,b'@lucywiryono Terima kasih mbak atas tread pe...
5,b'yang sibuk ngurusin hal2 diluar isi nya mema...
6,b'RT @apnmoa: Pada tahun 1997 @Anggun_Cipta m...
7,b'RT @aniesbaswedan: Hujan deras tidak menyuru...
8,b'RT @aniesbaswedan: Hujan deras tidak menyuru...
9,b'#MotoGPMandalika orang indo sekali nonton ko...


In [3]:
#definisi dataframe
df = pd.DataFrame(df_twit[['teks']])

#menampilkan dataframe
df.head(10)

,teks
0,"b""RT @Rizmaya__: Semula Mencibir, Kini Terpana..."
1,b'@Diniandik @lucywiryono Sabar mbak gitu dah ...
2,b'RT @aniesbaswedan: Hujan deras tidak menyuru...
3,"b'Dari Mandalika, Presiden @jokowi dan Ibu Iri..."
4,b'@lucywiryono Terima kasih mbak atas tread pe...
5,b'yang sibuk ngurusin hal2 diluar isi nya mema...
6,b'RT @apnmoa: Pada tahun 1997 @Anggun_Cipta m...
7,b'RT @aniesbaswedan: Hujan deras tidak menyuru...
8,b'RT @aniesbaswedan: Hujan deras tidak menyuru...
9,b'#MotoGPMandalika orang indo sekali nonton ko...


In [4]:
#menghilangkan mention/user
def remove_pattern(tweet, pattern):
    r = re.findall(pattern, tweet)
    for i in r:
        tweet = re.sub(i, '', tweet)
    return tweet    
df['remove_user'] = np.vectorize(remove_pattern)(df['teks'], "@[\w]*")

In [5]:

def tweet_clean(tweet):
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
    
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'RT :[\s]+', '', tweet)
    
     # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
     # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    #Happy Emoticons
    emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
    
    #Sad Emoticons
    emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
    #all emoticons (happy + sad)
    emoticons = emoticons_happy.union(emoticons_sad)
              
    #tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) #stemming word
            tweets_clean.append(stem_word)
    return tweets_clean
df['tweet_clean'] = df['remove_user'].apply(lambda x: tweet_clean(x))

In [6]:
df.head(10)

,teks,remove_user,tweet_clean
0,"b""RT @Rizmaya__: Semula Mencibir, Kini Terpana...","b""RT : Semula Mencibir, Kini Terpana dgn 'Kesa...","[b, cibir, terpana, dgn, sakti, pawang, hujan,..."
1,b'@Diniandik @lucywiryono Sabar mbak gitu dah ...,b' Sabar mbak gitu dah ga nonton tp ngata2in ...,"[b, sabar, mbak, gitu, dah, ga, nonton, tp, ng..."
2,b'RT @aniesbaswedan: Hujan deras tidak menyuru...,b'RT : Hujan deras tidak menyurutkan semangat ...,"[b hujan, deras, surut, semangat, ribu, tonton..."
3,"b'Dari Mandalika, Presiden @jokowi dan Ibu Iri...","b'Dari Mandalika, Presiden dan Ibu Iriana mel...","[b dari, mandalika, presiden, iriana, lanjut, ..."
4,b'@lucywiryono Terima kasih mbak atas tread pe...,b' Terima kasih mbak atas tread pencerahannya ...,"[b, terima, kasih, mbak, tread, cerah, kait, m..."
5,b'yang sibuk ngurusin hal2 diluar isi nya mema...,b'yang sibuk ngurusin hal2 diluar isi nya mema...,"[b yang, sibuk, ngurusin, luar, isi, nya, tont..."
6,b'RT @apnmoa: Pada tahun 1997 @Anggun_Cipta m...,"b'RT : Pada tahun 1997 mengenalkan "" Pawang ...","[b pada, kenal, pawang, hujan, dunia, lagu, ju..."
7,b'RT @aniesbaswedan: Hujan deras tidak menyuru...,b'RT : Hujan deras tidak menyurutkan semangat ...,"[b hujan, deras, surut, semangat, ribu, tonton..."
8,b'RT @aniesbaswedan: Hujan deras tidak menyuru...,b'RT : Hujan deras tidak menyurutkan semangat ...,"[b hujan, deras, surut, semangat, ribu, tonton..."
9,b'#MotoGPMandalika orang indo sekali nonton ko...,b'#MotoGPMandalika orang indo sekali nonton ko...,"[b motogpmandalika, orang, indo, nonton, komen..."


In [7]:
#remove punct
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text
df['Tweet'] = df['tweet_clean'].apply(lambda x: remove_punct(x))

In [8]:
df.head(10)

,teks,remove_user,tweet_clean,Tweet
0,"b""RT @Rizmaya__: Semula Mencibir, Kini Terpana...","b""RT : Semula Mencibir, Kini Terpana dgn 'Kesa...","[b, cibir, terpana, dgn, sakti, pawang, hujan,...",b cibir terpana dgn sakti pawang hujan motogpm...
1,b'@Diniandik @lucywiryono Sabar mbak gitu dah ...,b' Sabar mbak gitu dah ga nonton tp ngata2in ...,"[b, sabar, mbak, gitu, dah, ga, nonton, tp, ng...",b sabar mbak gitu dah ga nonton tp ngatain yg ...
2,b'RT @aniesbaswedan: Hujan deras tidak menyuru...,b'RT : Hujan deras tidak menyurutkan semangat ...,"[b hujan, deras, surut, semangat, ribu, tonton...",b hujan deras surut semangat ribu tonton tribu...
3,"b'Dari Mandalika, Presiden @jokowi dan Ibu Iri...","b'Dari Mandalika, Presiden dan Ibu Iriana mel...","[b dari, mandalika, presiden, iriana, lanjut, ...",b dari mandalika presiden iriana lanjut kunjun...
4,b'@lucywiryono Terima kasih mbak atas tread pe...,b' Terima kasih mbak atas tread pencerahannya ...,"[b, terima, kasih, mbak, tread, cerah, kait, m...",b terima kasih mbak tread cerah kait motogpman...
5,b'yang sibuk ngurusin hal2 diluar isi nya mema...,b'yang sibuk ngurusin hal2 diluar isi nya mema...,"[b yang, sibuk, ngurusin, luar, isi, nya, tont...",b yang sibuk ngurusin luar isi nya tonton musi...
6,b'RT @apnmoa: Pada tahun 1997 @Anggun_Cipta m...,"b'RT : Pada tahun 1997 mengenalkan "" Pawang ...","[b pada, kenal, pawang, hujan, dunia, lagu, ju...",b pada kenal pawang hujan dunia lagu judul plu...
7,b'RT @aniesbaswedan: Hujan deras tidak menyuru...,b'RT : Hujan deras tidak menyurutkan semangat ...,"[b hujan, deras, surut, semangat, ribu, tonton...",b hujan deras surut semangat ribu tonton tribu...
8,b'RT @aniesbaswedan: Hujan deras tidak menyuru...,b'RT : Hujan deras tidak menyurutkan semangat ...,"[b hujan, deras, surut, semangat, ribu, tonton...",b hujan deras surut semangat ribu tonton tribu...
9,b'#MotoGPMandalika orang indo sekali nonton ko...,b'#MotoGPMandalika orang indo sekali nonton ko...,"[b motogpmandalika, orang, indo, nonton, komen...",b motogpmandalika orang indo nonton komen nya ...


In [9]:
df.sort_values('Tweet', inplace = True)
df.drop(df.columns[[0,1]], axis = 1, inplace = True)
df.drop_duplicates(subset ='Tweet', keep = 'first', inplace = True)
df.to_csv('mandalika21032022_edit1_cleanbobot.csv',encoding='utf8', index=False)
df.head(10)

,tweet_clean,Tweet
6149,[b],b
2077,"[b aakk, tuju, bgt, lgsg, jatuh, hati, xf, xf,...",b aakk tuju bgt lgsg jatuh hati xf xf x x xf x...
5005,"[b acara, motogp, mandalika, ubah, cek, update...",b acara motogp mandalika ubah cek update motog...
812,"[b ada, aja, fq, n, nmotogp, fabioquartararo, ...",b ada aja fq n nmotogp fabioquartararo fq moto...
139,"[b ada, data, tarik, statistik, juara, dunia, ...",b ada data tarik statistik juara dunia motogp ...
3888,"[b ada, event, motogp, jalan, sirkuit, mandali...",b ada event motogp jalan sirkuit mandalika pad...
326,"[b ada, gak, sih, pawang, koruptor, pawang, ol...",b ada gak sih pawang koruptor pawang oligarki ...
4976,"[b ada, kenal, baju, putih, senggol, n, nindon...",b ada kenal baju putih senggol n nindonesiangp...
762,"[b ada, malu, pawang, hujan, motogpmandalika, ...",b ada malu pawang hujan motogpmandalika n nbar...
4090,"[b ada, motogpmandalika, jalan, lengang, mante...",b ada motogpmandalika jalan lengang mantengin tv
